Notebook esame: 
In questo notebook mi occuperò dell' analisi degli spettri di assorbimento di due specie di molecole (CO2 e C2H2), ottenuti attraverso il metodo 'comb-locked frequency-swept synthesis' (CLFSS), il quale permette di avere simultanaemante un grande livello di precisione e di range di misura in tempi brevi.
Come primo passo importo le librerie necessarie.

In [1]:
# Import libraries
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
#import ipywidgets as widgets
#from ipywidgets import interact_manual
#from ipywidgets import interact
from scipy.signal import find_peaks
import spettro_module as sm

Ora carico i dati attraverso la funzione loadtxt della libreria numpy, osservando i dati è possibile individuare una baseline, la quale viene subito sottratta ai dati

In [2]:
# Dizionario dei dati
dati = {}
bline=[.005,2.5e-9]
# Costruisco i vettori
myList=['C2H2', 'CO2'] #modifica in percorsi relativi
dati['C2H2']=np.loadtxt("ATLAS_2018/C2H2_Pband_10Torr.dat",delimiter="\t", skiprows=2, usecols=range(6))
dati['CO2']=np.loadtxt("ATLAS_2018/CO2_3THz_scan_CRDS__data_2e-2Torr.dat",delimiter="\t", skiprows=2,usecols=range(6))
for i,name in enumerate(myList):
    dati[name][:,-1]=dati[name][:,-1]-bline[i]


In [3]:
#prova plot dati
plt.close('all')

for name in myList:
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1],label=name)
    plt.legend()
    plt.grid()


Ora come prima analisi è utile trovare la posizione e l'altezza dei massimi dei picchi, i quali saranno l'oggetto d'interesse per la distinzione delle varie bande. Inoltre questi dati fungono da starting point per i fit di ogni picco. 

In [4]:
plt.close('all')
#algoritmo di peaks
#per ora lasciamo da parte il calcolo della baseline
peaks={}
hmin=[.02, 6e-9]
width=10
for i,name in enumerate(myList):
    peaks[name]=[dati[name][find_peaks(dati[name][:,-1],height=hmin[i], width=width)[0],0],dati[name][find_peaks(dati[name][:,-1],height=hmin[i], width=width)[0],-1]]
    plt.figure()
    plt.plot(dati[name][:,0],dati[name][:,-1])
    plt.plot(peaks[name][0],peaks[name][1], '.')
    plt.grid()


Ora si può procedere all' analisi in due metodi, che chiameremo 'fit dei picchi' e 'fit delle bande'. Il primo consiste nell'analizzare con un fit i singoli picchi di assorbimento, mentre per il secondo si procede con l'analisi dei massimi dei picchi appartenenti alla stessa banda. Iniziamo illustrando il primo metodo.

Per analizzare i picchi è necessario per prima cosa isolarli, per fare ciò inizio a togliere tutti i dati negativi, successivamente, utilizzando i massimi dei picchi trovati nella cella precedente come contatori, per ogni picco trovo un xmin e xmax, ovvero inizio e fine del picco con il minimo/massimo della funzione 1/(x-x_picco). Isolato il picco posso procedere con il fit, ma prima noto che alcuni picchi sono sovrapposti, di conseguenza non è possibile distinguerli e la funzione di fit non convergerà. Per eliminare tale problema, ho deciso di escludere dal fit tali picchi, attraverso un if. Per una migliore convergenza del fit, infine, ho scelto di utilizzzare starting points e bounds: per i primi, data la funzione di voigt definita nel modulo, ho utilizzato l'area sottesa da ogni picco trovata in approssimazione di integrale discreto di Riemann, e la Full Width at Half Maximum moltiplicata per alcuni fattori numerici; per i secondi mi sono avvalso di un vettore di controllo (ctrlv) per valutare l'errore percentuale del fit sul guess, e aggiustare i bounds di conseguenza. 

In [17]:
#fit dei picchi
plt.close('all')

ctrlv=[[],[]]
for i,name in enumerate(myList):
    plt.figure()
    plt.grid()
    cond=(dati[name][:,-1]<=0)
    dati2=dati[name][:,0]*cond
    plt.plot(dati[name][:,0],dati[name][:,-1],'k',linewidth=.1)
    for c,j in enumerate(peaks[name][0]):
        #perche` questa formula? voglio trovare il pt piu` vicino che sia sotto la bline (altrimenti arrivo al picco)`
        xmin=np.argmin(1/(dati2-j))
        xmax=np.argmax(1/(dati2-j)) 
        xfit=dati[name][xmin:xmax,0]
        yfit=dati[name][xmin:xmax,-1]
        #grazie all'if elimino i picchi sovrapposti
        if c!=len(peaks[name][0])-1:
            if np.logical_and(peaks[name][0][c+1]>xfit[-1],peaks[name][0][c-1]<xfit[0]):
                
                FWHM=xfit[yfit>=max(yfit)/2][-1]-xfit[yfit>=max(yfit)/2][0]

                #per il fit utilizzo la funzione di voigt
                p0=[np.dot(np.diff(xfit),yfit[:-1]),FWHM/(2*np.sqrt(2*np.log(2))),j,FWHM/2]
                fit,cov=curve_fit(sm.voigt,xfit,yfit,p0=p0,
                                  bounds=((p0[0]/2,p0[1]/2,p0[2]-.1,p0[3]*.09),(p0[0]*2,p0[1],p0[2]+.1,p0[3]*2)))
                #grazie a questo vettore controllo la bonta` del guess
                ctrlv[i].append((p0-fit)/p0)
                #print((p0-fit)/p0)
                #controllo che funzioni il metodo scelto
                #plt.plot(dati[name][:,0],1/(dati2-j))
                plt.plot(xfit,yfit,'r',linewidth=1)
                plt.plot(xfit,sm.voigt(xfit,*fit),'--b',linewidth=.5)
                plt.plot(xfit,sm.voigt(xfit,*p0),'--g',linewidth=.5)
    #nu Rolex or e diamant
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Assorbimento')
    plt.title(name)
    plt.legend(['whole profile','selected data','fit curve','guess'])


Si può notare come il guess non sia molto preciso, per migliorare la convergenza del fit si potrebbe fare uno studio più approfondito sulla teoria e sulla funzione di voigt oppure usare metodi iterativi

Illustriamo ora il secondo metodo, ovvero il fit delle bande. Per fittare il modello, ovvero una distribuzione di Maxwell-Boltzmann, occorre innanzitutto distinguere le bande, poiché per ora non lo sono. Per fare ciò osservo che dal plot dei massimi dei picchi, il profilo che ne risulta è a sua volta un profilo a picchi, quindi posso riutilizzare la funzione find_peaks per distinguere la banda più alta dal resto dei dati, utilizzando i parametri distance e hmin, scelti dall'utente, per selezionare solo i dati desiderati. Noto inoltre che questo metodo permette anche di dare una stima del centro della banda poiché esso è punto di minimo relativo 

In [6]:
#provo a plottare i massimi dei massimi e vedere se riconosce le bande
plt.close('all')
peaks2={}
hmin=[.15, 2e-8]
width=0
gmu=[0,0]
dist=[7,2] #,distance=dist
for i,name in enumerate(myList):
    peaks2[name]=[peaks[name][0][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]],peaks[name][1][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]]]
    #e` anche possibile dare cosi` una stima del centro delle bande
    gmu[i]=peaks2[name][0][find_peaks(-peaks2[name][1])[0]]
    plt.figure()
    plt.plot(peaks[name][0],peaks[name][1], 'r-*')
    plt.plot(peaks2[name][0],peaks2[name][1], 'b.')
    plt.plot(gmu[i],0,'g.')
    plt.grid()


Una volta distinta la banda superiore dal resto, noto che molti punti che dovrebbero appertenervi mancano. Questo perchè il metodo precedente riesce a distinguerli soltanto se rappresentano dei massimi, se si trovano in una zona in cui non sono presenti massimi di altre bande, questi non spiccheranno tra i dati, e quindi non verranno considerati. Per ovviare a ciò eseguo un fit preliminare di questa prima scrematura di bande, sempre utilizzando degli starting points trovati manipolando la funzione di riferimento, successivamente considero nuovamente tutti i massimi. Se questi distano dal modello meno di una certa ε, arbitraria ma in prima approssimazione uguale a circa  2,5 volte la deviazione standard dei dati selezionati dal modello, allora anche essi vengono considerati parte della banda. Ora che ho aggiunto alcuni dati, con un certo limite dato da una soglia in altezza (poiché al di sotto di essa vi sono troppe bande e quindi i dati sono troppo densi), eseguo nuovamente il fit, in modo da avere dei parametri migliori del modello, il quale di adatterà meglio ai dati.

In [20]:
#fit delle bande
plt.close('all')
kb=1.380649 *10**-23
#gmu=[196.8,190.29]
#idea: ciclo while che diminuisce eps ma lo tiene tale che tutti i selected point siano nella ricostruzione
#eps=[2e-1,3e-8]
vmax=[0,0]
norm=[0,0]
peaks2={}
hmin=[.15, 2e-8]
width=0
gmu=[0,0]
dist=[7,2]

for i,name in enumerate(myList):
    #qui potrei inserire un ciclo per fittare ogni banda
    #for j in range(#bande):
    peaks2[name]=[peaks[name][0][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]],peaks[name][1][find_peaks(peaks[name][1],height=hmin[i],distance=dist[i])[0]]]
    #e` anche possibile dare cosi` una stima del centro delle bande
    gmu[i]=peaks2[name][0][find_peaks(-peaks2[name][1])[0]]
    b_max=np.argmax(peaks2[name][1])
    plt.figure()
    plt.plot(peaks[name][0],peaks[name][1],'k.',markersize=2)
    for k in range(2):
        if peaks2[name][0][b_max]>gmu[i]:
            cond2=peaks2[name][0]>=gmu[i]
            cond3=peaks[name][0]>=gmu[i]
        else:
            cond2=peaks2[name][0]<=gmu[i]
            cond3=peaks[name][0]<=gmu[i]
        #lasciamo perdere l'approccio di matematica booleana
        if np.logical_and(i!=0,k==1):
            cond2=np.logical_not(cond2)
            cond3=np.logical_not(cond3)
        xb=np.copy(peaks2[name][0][cond2])
        yb=np.copy(peaks2[name][1][cond2])
        vmax[i]=abs(xb[np.argmax(yb)]-gmu[i])
        #vmax e` possibile trovarlo anche conoscendo T e m
        #vmax[1]=np.sqrt(kb*(8.21917808*10**21)) 
        #f(vmax)=C*... inverto la relazione
        norm[i]=np.exp(1)*yb[np.argmax(yb)]*abs(vmax[i])*(np.pi**(1/2))/4
        #si puo` provare anche approssimando ad un triangolo
        p0=[vmax[i],norm[i],gmu[i]]
        fit,cov=curve_fit(sm.max_boltz,xb,yb,p0=p0)
        #si potrebbe computare eps partendo ad esempio dai residui o dalla std, ho scelto 2.5 volte la std
        eps[i]=2.5*np.sqrt(np.mean((yb-sm.max_boltz(xb,*fit))**2))
        #ricostruzione banda #*abs(xb-xb[np.argmax(yb)]) #*abs(xb-xb[b_max])  
        #bcon=[for j in len(xb[bcon]):  ]
        bcon=((abs(peaks[name][1]-sm.max_boltz(peaks[name][0],*fit)))<=eps[i])*cond3*(peaks[name][1]>=hmin[i])
        fit2,cov2=curve_fit(sm.max_boltz,peaks[name][0][bcon],peaks[name][1][bcon],p0=p0)
        #plot
        plt.plot(xb,yb, '*b',markersize=7)
        plt.plot(peaks[name][0][cond3],sm.max_boltz(peaks[name][0][cond3],vmax[i],norm[i],gmu[i]),'g',linewidth=.5)
        plt.plot(peaks[name][0][cond3],sm.max_boltz(peaks[name][0][cond3],*fit),'grey',linewidth=.5)
        plt.plot(peaks[name][0][bcon],peaks[name][1][bcon],'.r')
        plt.plot(peaks[name][0][cond3],sm.max_boltz(peaks[name][0][cond3],*fit2),'k--',linewidth=1)

    #nu Rolex or e diamant
    plt.grid()
    plt.legend(['whole data','selected data','guess','1st fit','reconstructed band','2nd fit'])
    plt.xlabel('Frequenza [THz]')
    plt.ylabel('Assorbimento')
    plt.title(name)

Si può notare che la banda è asimmetrica, ma che in generale i dati si adattano piuttosto bene al modello scelto. Per migliorare il fit si potrebbe utlizzare la distinzione di banda trovata per fittare i risultati ottenuti dal fit dei picchi, poichè tali dati presentano un errore minore dato dal fit stesso e non dalla precisione dello strumento. Si potrebbe inoltre iterare nuovamente il processo per il fit delle bande inferiori per eseguire analisi più approfondite.